# Lattice movement
This notebook aims at providing tools for simulating the movement of particles in a lattice.

In [138]:
import numpy as np
from scipy import integrate
from numpy import pi, tan, log, exp, sqrt
import time as tm
from bokeh.plotting import figure, show, output_notebook, ColumnDataSource
from bokeh.io import push_notebook
from bokeh.layouts import row, gridplot
output_notebook()

Loading BokehJS ...

The "hat" function is defined as follows:
$$ \hat{x}:=Re(x)+Im(x) $$

In [2]:
def hat(z):
    return z.real + z.imag

Some general stuff

In [26]:
# Gives polar representation of z
def polar(z):
    a = log(complex(z))
    r = exp(a.real)
    theta = a.imag
    return r, theta

In [4]:
def deg(theta):
    return theta*360/(2*pi)

def rad(theta):
    return theta/360*2*pi

In [5]:
def inspect(z):
    print("The number:")
    print(f"{z:.2f}")
    print("In polar form:")
    z = polar(z)
    print(f"{z[0]:.2f}*e^(i*{z[1]:.2f})")
    print("Angle in degrees:")
    print(f"{deg(z[1]):.2f}")

In [6]:
# Saving values of recursive function for time efficiency gains
saved_data = dict()

def clear():
    global saved_data
    saved_data = dict()

def saved(func):
    h = hash(func)
    def f(*params):
        try:
            return saved_data[(h,*params)]
        except KeyError:
            val = func(*params)
            saved_data[(h,*params)] = val
            return val
    return f

Some $\psi$-distributions

In [7]:
def psi_gen(v, theta, optimize=True, diagonalized=True):
    # When diagonalized we the input v from [0;1] is mapped onto a v which induces an expected value speed in [0;sqrt(2)/2]
    if diagonalized:
        v = v*(1+tan(theta))/sqrt(2+2*tan(theta)**2)
    
    @saved
    def psi(x,t):
        # Optimization
        if optimize:
            if x.real < 0 or x.imag < 0 or hat(x)>t:
                return 0
        
        if t == 0:
            return 1 if x == 0 else 0
        else:
            return (psi(x-1, t-1) * v*1/(1 + tan(theta)) # propagation from the tile below
                    + psi(x-1j, t-1) * v*tan(theta)/(1 + tan(theta)) # propagation from the tile on the left
                    + (1-v)*psi(x, t-1)) # residue from the tile itself
                    # technically this formula is incomplete as motion with theta outside of the interval [0,pi] wouldn't work
    return psi

# Shorthand
gen = psi_gen

As we can't sum over the whole plane, we have to limit ourselves to a subset of $\mathbb{C}$. This doesn't change the result of the calculation in most cases though, because $\psi$ will be zero outside our range anyways. We will choose a range from 0 to 100 for the imaginary and real part respectively.

In [210]:
subset = list(np.ndindex((10,10)))

In [84]:
def expected_value(psi,t):
    return sum(psi(complex(a,b),t)*complex(a,b) for a, b in subset)

# Shorthand
ex = expected_value

In [79]:
def variance(psi, t):
    e = ex(psi, t)
    return sum(psi(complex(a,b),t)*abs(complex(a,b)-e)**2 for a, b in subset)

# Shorthand
var = variance

## First observation
The maximum speed gets lower the more diagonal the angle gets!

In [85]:
x = [float(x) for x in np.linspace(0,90,20)]
y_diag = []
y_disc = []

for i in x:
    e_diag = ex(psi_gen(1, rad(i), diagonalized=True),1)
    e_disc = ex(psi_gen(1, rad(i), diagonalized=False),1)
    y_diag.append(abs(e_diag))
    y_disc.append(abs(e_disc))
    
p = figure()
p.line(x,y_diag)
p.line(x,y_disc)
show(p)

In [80]:
clear()

In [11]:
inspect(ex(gen(1, rad(13.4543)), 1))

The number:
0.69+0.16j
In polar form:
0.71*e^(i*0.23)
Angle in degrees:
13.45


## Plot movement

In [13]:
# Returns list of vals
def vals(psi, N, time=None):
    return [psi(complex(a,b), time if time is not None else a+b) for a in range(N) for b in range(N)]

In [14]:
# Maps an array of numbers onto a color scale
black_color_scale = ["#" + f"{int(i):02x}"*3 for i in np.linspace(255,0,101)]
red_color_scale = ["#" + f"{int(i):02x}"*1 + "0000" for i in np.linspace(255,0,101)]

def to_colors(array, low=None, high=None, red=False):
    if low is None:
        low = min(array)
    if high is None:
        high = max(array)
    if not red:
        return [black_color_scale[int((array[i]-low)/(high-low)*100)] for i in range(len(array))]
    else:
        return [red_color_scale[int((array[i]-low)/(high-low)*100)] for i in range(len(array))]

In [15]:
to_colors(range(5,12),low=0,high=15)

['#aaaaaa', '#999999', '#898989', '#777777', '#666666', '#565656', '#444444']

In [16]:
psi = psi_gen(1,rad(60))

N = 5
time = 4

r =[f"{i}" for i in range(N)]
x = [f"{int(i/N)}" for i in range(N*N)]
y = [f"{i}" for i in range(N)]*N

p = []
for t in range(5):
    p.append(figure(title=f"t={t}", toolbar_location=None, x_range=r, y_range=r,
            width=195, height=210))

    p[-1].rect(x, y, color=to_colors(vals(psi,N,time=t)), width=1, height=1)

show(row(*p))

In [17]:
def plot(func, time, N):
    
    r =[f"{i}" for i in range(N)]
    x = [f"{int(i/N)}" for i in range(N*N)]
    y = [f"{i}" for i in range(N)]*N

    p = []
    
    if not hasattr(time, '__iter__'):
        p = figure(title=f"t={time if time is not None else '<all>'}", toolbar_location=None, x_range=r, y_range=r,
                width=500, height=500)
        p.rect(x, y, color=to_colors(vals(func, N, time=time)), width=1, height=1)
        show(p)
    else: 
        for t in list(time):
            p.append(figure(title=f"t={t if t is not None else '<all>'}", toolbar_location=None, x_range=r, y_range=r,
                    width=195, height=210))

            p[-1].rect(x, y, color=to_colors(vals(func, N, time=t)), width=1, height=1)

        splitted = [[p[i+n*5] for i in range(min(5,len(p)-n*5))] for n in range(int((len(p)+4)/5))]

        show(gridplot(*splitted, toolbar_location=None))

In [219]:
plot(psi, list(range(9))+[None], 10)

In [19]:
plot(gen(1,rad(20)),None,20)

In [20]:
inspect(ex(gen(1,rad(20)), 20))

The number:
13.29+4.84j
In polar form:
14.14*e^(i*0.35)
Angle in degrees:
20.00


In [21]:
def animated(func, time, N, end_after=None):
    r =[f"{i}" for i in range(N)]
    x = [f"{int(i/N)}" for i in range(N*N)]
    y = [f"{i}" for i in range(N)]*N

    p = figure(title=f"t=<all>", toolbar_location=None, x_range=r, y_range=r,
            width=500, height=500)
    r = p.rect(x, y, color=to_colors(vals(func, N, time=0)), width=1, height=1)

    show(p, notebook_handle=True)

    t = 1
    while True:
        tm.sleep(0.01)

        r.data_source.data['fill_color'] = to_colors(vals(func, N, time=t))
        r.data_source.data['line_color'] = to_colors(vals(func, N, time=t))
        
        push_notebook()

        t = (t + 1) % time
        
        # For stopping the loop at the <end_after>th frame
        if not end_after is None:
            end_after -= 1
            if end_after == 1:
                break

In [22]:
animated(gen(1, rad(0)), 20, 20, end_after=20)

In [23]:
psi_1= gen(1, rad(0))
psi_2 = gen(1, rad(45))
psi_3 = gen(1, rad(70))

psi_n1 = gen(1, rad(0), diagonalized=False)
psi_n2 = gen(1, rad(45), diagonalized=False)
psi_n3 = gen(1, rad(70), diagonalized=False)


t = [int(i) for i in np.linspace(0, 20, 20)]

y_1 = [var(psi_1, i) for i in t]
y_2 = [var(psi_2, i) for i in t]
y_3 = [var(psi_3, i) for i in t]

y_n1 = [var(psi_n1, i) for i in t]
y_n2 = [var(psi_n2, i) for i in t]
y_n3 = [var(psi_n3, i) for i in t]

In [24]:
p = figure()
p.line(t, y_1, color="red", line_width=3, legend="0 deg")
p.line(t, y_2, color="green", line_width=3, legend="45 deg")
p.line(t, y_3, line_width=3, legend="70 deg")
p.line(t, y_n1, color="red", line_width=1, legend="0 deg (no diag)")
p.line(t, y_n2, color="green", line_width=1, legend="45 deg (no diag)")
p.line(t, y_n3, line_width=1, legend="70 deg (no diag)")
show(p)

In [25]:
theta = [int(i) for i in np.linspace(0,90,30)]
y = [var(gen(1, rad(i)), 1) for i in theta]
y_n = [var(gen(1, rad(i), diagonalized=False), 1) for i in theta]

p = figure()
p.line(theta, y, line_width=3, legend="with diagonalization")
p.line(theta, y_n, legend="without diagonalization")
show(p)

### Problem
We don't like that the variance depends on the angle; it should be equal for every angle. In order to achieve that, we have to introduce another model. 

Not assuming that it will resolve the problem for sure, we will later try to utilize complex variables as the range for our wave function. The probability density function of position is than the absolute value squared of this function. In that case the velocity of our particle is given by the Fourier Transform of the position wave function. Therefore, we first of all implement the fourier transform for the real case.

## Discrete-time Fourier Transform

We have
$$\hat{\Psi}(\pmb{\omega}) = \sum_{\pmb{n}\in\mathbb{Z}^2}\Psi(\pmb{n})e^{-i(\pmb{\omega}\cdot\pmb{n})}$$
and the inverse
$$\Psi(\pmb{n})=\frac{1}{4\pi^2}\int^{2\pi}_0\int^{2\pi}_0 \hat{\Psi}(\pmb{\omega})e^{i(\pmb{\omega}\cdot\pmb{n})}\operatorname{d}\pmb{\omega}.$$
Let us first of all implement the former. Instead of summing over the whole plane, we will only sum over a certain range -- as we did for the expected value and the variance --, since the values are zero outside of that range which leads to the respective terms canceling anyways.

In [108]:
def dtft_gen(psi):
    def psi_hat(omega,t):
        return sum(psi(complex(a,b),t) * exp(-1j*(omega[0]*a+omega[1]*b)) for a,b in subset)
    return psi_hat

In [167]:
def inverse_dtft(psi_hat):
    def psi(x,t):
        def real_integrand(w1,w2):
            integrand = psi_hat((w1,w2),t)*exp(1j*(w1*x.real+w2*x.imag))
            return integrand.real
        def imag_integrand(w1,w2):
            integrand = psi_hat((w1,w2),t)*exp(1j*(w1*x.real+w2*x.imag))
            return integrand.imag
        lower_bound = lambda x: 0
        upper_bound = lambda x: 2*pi
        return (1/(4*pi**2)*integrate.dblquad(real_integrand, 0, 2*pi, lower_bound, upper_bound)[0]**2
                + 1/(4*pi**2)*integrate.dblquad(imag_integrand, 0, 2*pi, lower_bound, upper_bound)[0]**2)
    return psi

The inverse DTFT is incredibly slow, which is why we use the DFT for practical purposes.

In [206]:
N = (10,10)

def dft_gen(psi):
    def psi_hat(k,t):
        if type(k) is complex:
            k = (k.real,k.imag)
        return sum(psi(complex(a,b),t) * exp(-2j*pi*(k[0]*a/N[0]+k[1]*b/N[1])) for a,b in list(np.ndindex(N)))
    return psi_hat

def inverse_dft(psi_hat):
    def psi(x,t):
        return 1/N[0]*sum(psi_hat((k0,k1),t) * exp(2j*pi*(k0*x.real/N[0]+k1*x.imag/N[1])) for k0,k1 in list(np.ndindex(N)))
    return psi

In [208]:
psi = gen(0.8,rad(20))
psi_hat = dft_gen(psi)
psi_i = inverse_dft(psi_hat)

In [211]:
for t in range(10):
    print(variance(psi,t))
    print(variance(psi_hat,t))

0.0
(39695700+0j)
0.405046229629
(1123346.11881-21569.6854339j)
0.810092459259
(59945.8323423-862.759327551j)
1.21513868889
(4257.64946817+1664.45855234j)
1.62018491852
(-367.339569758+1104.02112654j)
2.02523114815
(-635.116246397+540.184394j)
2.43027737778
(-488.360456405+222.302162691j)
2.83532360741
(-344.567880959+59.7179949382j)
3.24036983703
(-239.806231216-23.0832374885j)
3.64541606666
(-165.575631833-69.8219956014j)


In [222]:
plot(lambda x,t: abs(psi_hat(x,t))**2, [1,2,3,4,5,6,7,8,9,10], 10)

In [223]:
plot(lambda x,t: psi_hat(x,t).real, range(1,11), 10)

In [224]:
plot(lambda x,t: psi_hat(x,t).imag, range(1,11), 10)

In [226]:
plot(lambda x,t: abs(psi_i(x,t))**2, range(1,11), 10)

In [228]:
plot(psi, range(1,11), 10)